# (Elbers, 2023) Encouragement Designs and Instrumental Variables for A/B Testing

[Link to Article](https://engineering.atspotify.com/2023/08/encouragement-designs-and-instrumental-variables-for-a-b-testing/)

# Introduction

 The core idea of an encouragement design is to assign the treatment to the entire population that we're testing on, but randomize the encouragement to use the feature.

 Compared to other observational causal techniques, an encouragement design has the advantage that it is still somewhat based on randomization, but does estimate a different quantity compared to a standard A/B test and requires a few assumptions.

# Three Types of Experiments

Before proceeding, some definitions:
- $Z$ is an indicator variable that indicates whether a uesr was assigned to treatment or control
- $D$ indicates whether a user was treated (actually did treatment)
- $Y$ is the outcome

## Full Compliance

In an ideal A/B test, we randomly assign each user to a treatment or control. All users in treatment group experience treatment, and all users in the control do not experience treatment.

Gold standard because we have only one group of users, which we call "compliers".

In this case,

$$E[Y|Z=1] - E[Y|Z=0]$$

identifies the ATE, the causal effect of the feature on the outcome.

## One-sided noncompliance

For instance, when launching a new feature, in the treatment we have users who actually used the feature "compliers" and those who didn't "noncompliers".

Now, the quantity $E[Y|Z=1] - E[Y|Z=0]$ no longer identifies the ATE, but is instead identifies the "intent-to-treat" effect, ITT.

While the ITT can be valuable, the ITT is usually much smaller than the causal effect of the feature due to dilution.

With compliance issues, we can't recover the ATE, but under certain assumptions we can recover the ATE for the compliers:

$$ATE_{\text{compliers}}=\frac{ITT}{E[D|Z=1]}=\frac{E[Y|Z=1]-E[Y|Z=0]}{E[D|Z=1]}$$

In other words, the ITT divided by the proportion of compliers.

This specialized ATE is also known as the "Local Average Treatment Effect (LATE)" or the complier average causal effect (CACE). The estimator is known as an instrumental variables estimator.



# Encouragement Design

In an encouragement design, we go one step further and allow noncompliance in both treatment and control. The random assignment is now no longer about feature availability, but about an encouragement to use the feature.

This could be, for instance, a banner that is only available to the treatment group.

Now we have:
- Treatment $Z=1$:
  - Treated: Always-takers and compliers
  - Untreated: Never-takers and defiers
- Control $Z=0$:
  - Treated: Always-takers and defiers
  - Untreated: Never-takers and compliers

Note how in practice we can never tell which group a user belongs to, because we only observe one state of the world.

Given that we have four mutually exclusive groups of users, we can write the ITT as a weighted average of the ITT within the four groups:

$$ITT=(p_{\text{always-takers}}*ITT_{\text{always-takers}}) + (p_{\text{compliers}}*ITT_{\text{compliers}})+(p_{\text{defiers}}*ITT_{\text{defiers}})+(p_{\text{never-takers}}*ITT_{\text{never-takers}})$$

where $p_i$ refers to the proportion of group $i$.

Additionally, we now make three key assumptions:
1. There are no defiers (**monotonicity**), i.e., $p_{\text{defiers}}=0$
2. The encouragement doesn't effect the outcomes for always-takers and never-takers (**exclusion restriction**), i.e., $ITT_{\text{always-takers}}=ITT_{\text{never-takers}}=0$
3. The encouragement works (**relevance**), i.e., $E[D|Z=1] > E[D|Z=0]$

With these assumptions, the formula above can be rearranged to make:

$$ITT_{\text{compliers}}=ATE_{\text{compliers}}=\frac{ITT}{p_{\text{compliers}}}=\frac{E[Y|Z=1]-E[Y|Z=0]}{E[D|Z=1]-E[D|Z=0]}$$

We have now derived the instrumental variables (IV) estimator. Again, $ATE_{\text{compliers}}$ will be undefined if there are no compliers, because then we'd divide by zero.

Note that IV estimators typically have higher standard errors because our statistical power only comes from a subset of the population.

It is important to realize that there are compliers in both the treatment and control. The compliers in control are compliers in the sense that they would have taken the treatment *if* we had encouraged them.

Another issue that "compliers" are not a well-defined population. Someone can be a complier in one test but not in another test. Whether it's useful to make inferences about compliers is a point that is contested in the literature on instrumental variables.

# A closer look at the assumptions of the IV estimator

The standard assumptions that need to be satisfied in any IV study are the SUTVA and randomization of treatment (here, the instrument).

Given a DAG of the relationship of instrument $Z$ with treatment $D$, outcome $Y$, and confounders $C$ (sorry, not pictured here), we have three assumptions around $Z$ that need to be satisfied:

1. **Monotonicity**: instrument always has to work in the same direction for all individuals, in other words, no defiers. This assumption is actually quite plausible in many settings (or at least, easier to convince that this is not a huge issue)
2. **Exclusion restriction**: visually, no path between $Z$ and $Y$ except through the feature $D$. Another way to state this is that the encouragement itself should not have an independent causal effect on the outcome $Y$. A violation of the exclusion restriction is not unlikely if the encouragement is very intrusive, especially for the never-takers who were not interested in the feature in the first place. In practice, the exclusion restriction is often the biggest issue in defending the interpretability of an IV estimate.
3. **Relevance of instrument**: direct arrow between $Z$ and $D$. Easily testable. It is also important that the effect of $Z$ on $D$ is numerically not too small, that is, the encouragement should be substantively relevant as well. If that is not the case, you run into the "weak instrument problem" which leads to larger standard errors and more seriously, a large bias in the IV estimator.



